# Simple example of time domain beamforming

Let's imagine that we have one or more idealised point sound sources with spherical spreading impinging upon an n-element linear microphone array.

The situation is depicted in the figures below.

Note that we are using point sources, not the plane-wave approximation.

We have used the following helpful links in developing this example:

* [Jupyter User guide](https://docs.bokeh.org/en/latest/docs/user_guide/jupyter.html)
* https://en.wikipedia.org/wiki/Wavelength
* [Equations for Plane Waves, Spherical Waves, and Gaussian Beams](https://onlinelibrary.wiley.com/doi/pdf/10.1002/9781118939154.app3)
* https://github.com/bokeh/bokeh/blob/1.3.2/examples/howto/layouts/dashboard.py
* https://onlinelibrary.wiley.com/doi/10.1002/9781119293132.ch11


For a linear array, a delay of 0 implies a beam steering direction of broadside (perpendicular
to the array).
A delay equivalent to the spacing between two adjacent elements divided by the velocity of
the soundwave implies a beam steering direction of either endfire (parallel to the array),
where the endfire (front or back) is determined by the sign of the delay

More generally, a delay of $d_t = \cos(\theta) \frac{d}{c}$ corresponds to a beam-steer at
an angle $\theta$ for $\theta \in [0^o, 180^o]$.

Given that we are dealing with discrete time signals, the delay cannot be chosen arbitrarily.

Instead, ...


## Import modules

In [ ]:
from ipywidgets import interact
import numpy as np

from bokeh.io import push_notebook, show, output_notebook
from bokeh.plotting import figure
from bokeh.models import Label
from bokeh.layouts import grid, column, gridplot
from bokeh.models import CustomJS, Slider, ColumnDataSource

import seaborn as sns
output_notebook()

## Set up parameters

In [ ]:
# Array characteristics
num_elements = 6
spacing = 0.1 # metres
x_location = 0.5 # metres
array_angle = np.deg2rad(45) # degrees
Arr = []
for k in range(num_elements):
    x_pos = x_location + k*spacing*np.sin(array_angle)
    y_pos = (k-(num_elements-1)/2)*spacing*np.cos(array_angle)
    Arr.append([x_pos,y_pos])
Arr = np.array(Arr)
col = sns.color_palette(None, num_elements).as_hex()


In [ ]:
# Source characteristics
num_sources = 1
Src = {}
Src['position'] = []
Src['amplitude'] = []
Src['wavelength'] = []
for n in range(num_sources):
    Src['position'].append([-1,0.5*n]) # Source position
    Src['amplitude'].append(1) # Source amplitude
    Src['wavelength'].append(0.1) # Source wavelength

#Src['amplitude'][1] = 0

## Create the Display

In [ ]:
π = np.pi
c = 340 # m/s (Speed of sound)

sample_rate = 96000

# For simplicity, and to avoid windowing issues, we choose a sample
# length which is an integer multiple of the period of the wavelength
# of the (first) source 
sample_length = 2*Src['wavelength'][0]/c # seconds

t_0 = 0

N = 500
x_min = -1
x_max = 1
x = np.linspace(x_min, x_max, N)
y = np.linspace(x_min, x_max, N)
t = np.arange(t_0, t_0+sample_length, 1/sample_rate)
xx, yy = np.meshgrid(x, y)

Z = 0*xx
for n in range(num_sources):
    [s_x, s_y] = Src['position'][n] #location of source
    A = Src['amplitude'][n]
    λ = Src['wavelength'][n] # metres
    f = c/λ # Hz

    print(f'Source {n+1}: A = {A}, ',
          f'f = {f} Hz, λ = {λ} m , T = {1/f:0.2e} s') 

    rr = np.sqrt((xx-s_x)*(xx-s_x) + (yy-s_y)*(yy-s_y))
        
    Z = Z + A/rr*np.sin(2*π/λ*(rr - c*t_0))

# Spacial Window
p = figure(tooltips=[("x", "$x"), ("y", "$y"), ("value", "@image")],
          )
p.xaxis.axis_label = p.yaxis.axis_label = 'Distance (m)'
p.x_range.range_padding = p.y_range.range_padding = 0

# must give a vector of image data for image parameter
p.image(image=[Z],
        x=x_min, y=x_min,
        dw=(x_max-x_min), dh=(x_max-x_min), 
        palette="Spectral11", level="image")

for k in range(num_elements):
    p.circle(Arr[k,0],Arr[k,1],
             size=10,
             fill_color=col[k],
             legend_label=f'Array Element {k}',
             )

p.grid.grid_line_width = 0.5
p.legend.click_policy="hide"

# Time Series Window
q = figure()
q.xaxis.axis_label = 'Time (s)'
q.yaxis.axis_label = 'Pressure'

Y = []
for k in range(num_elements):
    F = 0*t
    for n in range(num_sources):
        [s_x, s_y] = Src['position'][n] #location of source
        A = Src['amplitude'][n]
        λ = Src['wavelength'][n] # metres
        f = c/λ # Hz
    
        r = np.sqrt((Arr[k,0]-s_x)*(Arr[k,0]-s_x) + 
                    (Arr[k,1]-s_y)*(Arr[k,1]-s_y))
        F = F + A/r*np.sin(2*π/λ*(r - c*t))
    Y.append(F)
    q.line(t,F,color=col[k],legend_label=f'Array Element {k}')

q.legend.click_policy="hide"

# Widget Window
amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)

# Beamforming Window

b = figure()


show(gridplot([p, q, widgets, b], ncols=2), notebook_handle=True)

In [ ]:
import numpy as np

from bokeh.layouts import grid, column
from bokeh.models import CustomJS, Slider, ColumnDataSource
#from bokeh.plotting import figure, output_file, show

tools = 'pan'


def bollinger():
    # Define Bollinger Bands.
    upperband = np.random.randint(100, 150+1, size=100)
    lowerband = upperband - 100
    x_data = np.arange(1, 101)

    # Bollinger shading glyph:
    band_x = np.append(x_data, x_data[::-1])
    band_y = np.append(lowerband, upperband[::-1])

    p = figure(x_axis_type='datetime', tools=tools)
    p.patch(band_x, band_y, color='#7570B3', fill_alpha=0.2)

    p.title.text = 'Bollinger Bands'
    p.title_location = 'left'
    p.title.align = 'left'
    p.plot_height = 600
    p.plot_width = 800
    p.grid.grid_line_alpha = 0.4
    return [p]


def slider():
    x = np.linspace(0, 10, 100)
    y = np.sin(x)

    source = ColumnDataSource(data=dict(x=x, y=y))

    plot = figure(
        y_range=(-10, 10), tools='', toolbar_location=None,
        title="Sliders example")
    plot.line('x', 'y', source=source, line_width=3, line_alpha=0.6)

    amp_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Amplitude")
    freq_slider = Slider(start=0.1, end=10, value=1, step=.1, title="Frequency")
    phase_slider = Slider(start=0, end=6.4, value=0, step=.1, title="Phase")
    offset_slider = Slider(start=-5, end=5, value=0, step=.1, title="Offset")

    callback = CustomJS(args=dict(source=source, amp=amp_slider, freq=freq_slider, phase=phase_slider, offset=offset_slider),
                        code="""
        const data = source.data;
        const A = amp.value;
        const k = freq.value;
        const phi = phase.value;
        const B = offset.value;
        const x = data['x']
        const y = data['y']
        for (var i = 0; i < x.length; i++) {
            y[i] = B + A*Math.sin(k*x[i]+phi);
        }
        source.change.emit();
    """)

    amp_slider.js_on_change('value', callback)
    freq_slider.js_on_change('value', callback)
    phase_slider.js_on_change('value', callback)
    offset_slider.js_on_change('value', callback)

    widgets = column(amp_slider, freq_slider, phase_slider, offset_slider)
    return [widgets, plot]


def linked_panning():
    N = 100
    x = np.linspace(0, 4 * np.pi, N)
    y1 = np.sin(x)
    y2 = np.cos(x)
    y3 = np.sin(x) + np.cos(x)

    s1 = figure(tools=tools)
    s1.circle(x, y1, color="navy", size=8, alpha=0.5)
    s2 = figure(tools=tools, x_range=s1.x_range, y_range=s1.y_range)
    s2.circle(x, y2, color="firebrick", size=8, alpha=0.5)
    s3 = figure(tools='pan, box_select', x_range=s1.x_range)
    s3.circle(x, y3, color="olive", size=8, alpha=0.5)
    return [s1, s2, s3]

l = grid([
    bollinger(),
    slider(),
    linked_panning(),
],)# sizing_mode='stretch_both')

show(l)